## Calculate grit with different normalization schemes

We compare whole well vs. control-based normalization in grit calculations.

In [1]:
import pathlib
import pandas as pd

from pycytominer.cyto_utils import infer_cp_features, output
from pycytominer.operations import get_na_columns

from cytominer_eval import evaluate
from cytominer_eval.transform import metric_melt
from cytominer_eval.operations.util import assign_replicates

In [2]:
output_dir = "results"

In [3]:
# Load different normalized data
data_dir = pathlib.Path("../../0.download-data/data/cell-health/profiles")
plate_file = pathlib.Path(f"{data_dir}/cell_health_profiles_merged_wholeplate_normalized_featureselected.tsv.gz")

profile_df = pd.read_csv(plate_file, sep="\t")

features = infer_cp_features(profile_df)
meta_features = infer_cp_features(profile_df, metadata=True)

print(profile_df.shape)
profile_df.head()

(3456, 437)


,Metadata_Plate,Metadata_Well,Metadata_WellCol,Metadata_WellRow,Metadata_cell_line,Metadata_gene_name,Metadata_pert_name,Cells_AreaShape_Compactness,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,...,Nuclei_Texture_InverseDifferenceMoment_Mito_5_0,Nuclei_Texture_InverseDifferenceMoment_RNA_10_0,Nuclei_Texture_InverseDifferenceMoment_RNA_20_0,Nuclei_Texture_SumAverage_AGP_5_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_Mito_5_0,Nuclei_Texture_SumEntropy_RNA_20_0,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_Variance_ER_20_0
0,SQ00014614,A01,1,A,ES2,EMPTY,EMPTY,0.193423,0.810565,0.394067,...,-1.069082,-0.157315,-0.114324,2.222454,-1.883674,-0.661164,-2.079387,-0.879608,-0.586046,-1.565373
1,SQ00014614,A02,2,A,ES2,MCL1,MCL1-5,0.063269,0.179460,-0.621186,...,0.745802,1.866147,1.293618,2.372686,-0.420094,-0.140664,-0.323854,-0.878583,-0.928709,-0.102971
2,SQ00014614,A03,3,A,ES2,AKT1,AKT1-1,0.200543,-0.002573,-0.097555,...,-0.343634,1.280407,0.543693,1.818817,0.137862,0.158269,-0.055158,-0.989307,-0.640994,-0.277220
3,SQ00014614,A04,4,A,ES2,KRAS,KRAS-2B,-0.200653,0.398271,-0.255667,...,-0.413471,1.226423,1.032073,1.413421,-0.598619,-0.790145,-0.885389,-1.388938,-1.007342,-1.696331
4,SQ00014614,A05,5,A,ES2,AKT1,AKT1-2,0.291402,0.460302,0.235751,...,-1.326784,0.881351,0.784807,1.141495,-0.280980,-0.534104,-0.395320,-1.298397,-1.026182,-0.859397


In [4]:
# Define cell health constants
barcode_col = "Metadata_pert_name"
gene_col = "Metadata_gene_name"

replicate_group_grit = {
    "replicate_id": barcode_col,
    "group_id": gene_col
}

control_group_cut = ["Chr2", "Luc", "LacZ"]
control_group_pert = ["EMPTY"]

control_barcodes_cut = (
    profile_df.loc[
        profile_df[replicate_group_grit["group_id"]].isin(control_group_cut),
        replicate_group_grit["replicate_id"]
    ]
    .unique()
    .tolist()
)

control_barcodes_pert = (
    profile_df.loc[
        profile_df[replicate_group_grit["group_id"]].isin(control_group_pert),
        replicate_group_grit["replicate_id"]
    ]
    .unique()
    .tolist()
)

control_barcodes = {
    "cutting_control": control_barcodes_cut,
    "perturbation_control": control_barcodes_pert
}

control_barcodes

{'cutting_control': ['Chr2-1',
  'Chr2-4',
  'Chr2-5',
  'Chr2-2',
  'Luc-1',
  'LacZ-3',
  'Luc-2',
  'LacZ-2',
  'Chr2-3',
  'Chr2-6'],
 'perturbation_control': ['EMPTY']}

In [5]:
%%time
grit_results = []
for cell_line in profile_df.Metadata_cell_line.unique():
    for control_barcode in control_barcodes:
        for cor_method in ["pearson", "spearman"]:
            result = evaluate(
                profiles=profile_df.query("Metadata_cell_line == @cell_line"),
                features=features,
                meta_features=[barcode_col, gene_col],
                replicate_groups=replicate_group_grit,
                operation="grit",
                similarity_metric=cor_method,
                grit_control_perts=control_barcodes[control_barcode]
            ).assign(
                cell_line=cell_line,
                barcode_control=control_barcode,
                cor_method=cor_method
            )

            grit_results.append(result)
    
grit_results = pd.concat(grit_results).reset_index(drop=True)

print(grit_results.shape)
grit_results.head()

(1428, 6)
CPU times: user 50.1 s, sys: 4.39 s, total: 54.5 s
Wall time: 54.6 s


,perturbation,group,grit,cell_line,barcode_control,cor_method
0,AKT1-1,AKT1,0.379727,ES2,cutting_control,pearson
1,AKT1-2,AKT1,0.342610,ES2,cutting_control,pearson
2,ARID1B-1,ARID1B,0.268362,ES2,cutting_control,pearson
3,ARID1B-2,ARID1B,0.334005,ES2,cutting_control,pearson
4,ATF4-1,ATF4,0.999549,ES2,cutting_control,pearson


In [6]:
# Output results
output_dir = "results"
output_file = pathlib.Path(f"{output_dir}/cell_health_grit_wholeplatenormalized.tsv")

grit_results.to_csv(output_file, sep="\t", index=False)